In [ ]:
# default_exp calibrate

# Calibration Procedure

> summary

Every OpenHSI camera is unique and requires calibration before use. This module provides the abstractions to create the calibration data which are then used in operation. 


In [ ]:
#hide

# documentation extraction for class methods
from nbdev.showdoc import *

# unit tests using test_eq(...)
from fastcore.test import *

# monkey patching class methods using @patch
from fastcore.foundation import *
from fastcore.foundation import patch

# imitation of Julia's multiple dispatch using @typedispatch
from fastcore.dispatch import typedispatch

# bring forth **kwargs from an inherited class for documentation
from fastcore.meta import delegates

In [ ]:
#export

from fastcore.foundation import patch
from fastcore.meta import delegates
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d
from PIL import Image
from scipy.signal import decimate

from typing import Iterable, Union, Callable, List, TypeVar, Generic, Tuple, Optional
import datetime
import json
import pickle

In [ ]:
#export

from openhsi.data import *
from openhsi.capture import *

In [ ]:

"""
cam_prop.settings["longitude"] = -17.7
cam_prop.settings["latitude"] = 146.1
cam_prop.settings["datetime_str"] = "2021-05-26 03:26"
cam_prop.settings["altitude"] = 0.12
cam_prop.settings["radiosonde_station_num"] = 94299
cam_prop.settings["radiosonde_region"] = "pac"
cam_prop.settings["sixs_path"] = "assets/sixsV1.1"
"""

In [ ]:
#export

class SettingsBuilderMixin():
    
    def say_hello(self,):
        print("Hello!")

In [ ]:

#export

class SettingsBuilderMetaclass(type):
    def __new__(cls, clsname:str, cam_class, attrs) -> "Class":
        """Create a SettingsBuilder class based on your chosen `CameraClass`."""
        return super(SettingsBuilderMetaclass, cls).__new__(cls, clsname, (cam_class,SettingsBuilderMixin), attrs)
    

def create_settings_builder(clsname:str, cam_class:"Camera Class") -> "SettingsBuilder Class":
    """Create a `SettingsBuilder` class called `clsname` based on your chosen `cam_class`."""
    return type(clsname, (cam_class,SettingsBuilderMixin), {})
    
    

In [ ]:


SettingsBuilder = create_settings_builder("SettingsBuilder",SimulatedCamera)
sb = SettingsBuilder(txt_path="assets/cam_settings.txt",pkl_path="assets/cam_calibration.pkl")
sb.say_hello()

Hello!



@delegates()
class SettingsBuilder(CameraProperties):
    
    def __init__(self,cam_class:"custom camera class" = SimulatedCamera, **kwargs):
        """"""
        super().__init__(**kwargs)
        
        self.cam_class = cam_class
        with cam_class(**kwargs) as cam:
            cam.start_cam()
            img = cam.get_img()
            cam.stop_cam()
            
        
        

sb = SettingsBuilder(img_path="assets/rocky_beach.png", n_lines=1024, processing_lvl = 3)

SB2 = type("SB2",(SimulatedCamera,),{"hello":2})
sb2 = SB2()
sb2.hello